In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# 2 Fundamental Concepts

## 1. Stochastic Processes

A **stochastic process** is a family of time indexed random variables $X(w, t)$, where $w$ belongs to a sample space ($\Omega$) and  $t$ belongs to an index set ($\mathbb{R}, \mathbb{Z}, \mathbb{C}$). 

For afixed $t$, $X(w, t)$ is **arandom variable**. 

For a given $w$, $X(w, t)$, as a function of $t$, is called a **sample function** or **realization**. 

The population that consists of all possible realizations is called the ensemble in stochastic processes and time series analysis. Thus, a time series is a realization or sample function from a certain stochastic process. 

The mean function of the process 
$$\mu_t = \mathbb{E}X_t$$

The variance function of the process 
$$\sigma_t^2 = \mathbb{E}(X_t-\mu_t)^2$$

The covariance function between $X_t$ and $X_s$ 
$$\gamma(t, s) = \mathbb{E}(X_t-\mu_t)(X_s-\mu_s)$$

The correlation function between $X_t$ and $X_s$ 
$$\rho(t, s) = \frac{\gamma(t, s)}{\sqrt{\sigma_t^2}\sqrt{\sigma_s^2}}$$

## 2. The Autocovariance and Autocorrelation Functions